In [20]:
import sympy
from sympy import lambdify
import dill
dill.settings["recurse"] = True
from sympy import Rational as R
from tqdm.notebook import tqdm
import numpy as np

In [2]:
(
    reference_x,
    reference_y,
    reference_x_1,
    reference_y_1,
    reference_x_2,
    reference_y_2,
    reference_x_3,
    reference_y_3,
) = sympy.symbols("x y x_1 y_1 x_2 y_2 x_3 y_3")

(
    reference_v11_1, reference_v11_2,
    reference_v12_1, reference_v12_2, 
    reference_v21_1, reference_v21_2, 
    reference_v22_1, reference_v22_2, 
    reference_v31_1, reference_v31_2, 
    reference_v32_1, reference_v32_2, 
) = sympy.symbols("v_(1:4)(1:3)_(1:3)")
f_interpolation = sympy.Matrix(sympy.symbols("f_1:10"))

## Define basis function

In [3]:
p1, p2, p3 = sympy.symbols("p1:4")

q1 = p1**2 * (R(3) - R(2) * p1) + R(2) * p1 * p2 * p3
q1 = q1.subs(p3, R(1) - p1 - p2)

q2 = p2**2 * (R(3) - R(2) * p2) + R(2) * p1 * p2 * p3
q2 = q2.subs(p3, R(1) - p1 - p2)

q3 = p3**2 * (R(3) - R(2) * p3) + R(2) * p1 * p2 * p3
q3 = q3.subs(p3, R(1) - p1 - p2)

r1 = p1**2 * (p1 - R(1)) - p1 * p2 * p3
r1 = r1.subs(p3, R(1) - p1 - p2)

r2 = p2**2 * p1 + R(1, 2) * p1 * p2 * p3
r2 = r2.subs(p3, R(1) - p1 - p2)

r3 = p3**2 * p1 + R(1, 2) * p1 * p2 * p3
r3 = r3.subs(p3, R(1) - p1 - p2)

s1 = p1**2 * p2 + R(1, 2) * p1 * p2 * p3
s1 = s1.subs(p3, R(1) - p1 - p2)

s2 = p2**2 * (p2 - R(1)) - p1 * p2 * p3
s2 = s2.subs(p3, R(1) - p1 - p2)

s3 = p3**2 * p2 + R(1, 2) * p1 * p2 * p3
s3 = s3.subs(p3, R(1) - p1 - p2)

hermit_basis = [q3, r3, s3, q1, r1, s1, q2, r2, s2]

hermit_basis = sympy.Matrix([itm.subs(p1, reference_x).subs(p2, reference_y) for itm in hermit_basis])

## Define mapping function

In [5]:
monomial_basis = sympy.Matrix([
    1,
    reference_x,
    reference_y,
    reference_x**2,
    reference_x*reference_y,
    reference_y**2,
    reference_x**3,
    reference_x**2*reference_y,
    reference_x*reference_y**2,
    reference_y**3,
])

V = sympy.zeros(10, 10)

for i, basis in enumerate(monomial_basis):
    
    V[i, 0] = basis.subs({reference_x: 0, reference_y: 0})
    V[i, 1] = basis.subs({reference_x: 1, reference_y: 0})
    V[i, 2] = basis.subs({reference_x: 0, reference_y: 1})

    V[i, 3] = basis.diff(reference_x).subs({reference_x: 0, reference_y: 0})
    V[i, 4] = basis.diff(reference_y).subs({reference_x: 0, reference_y: 0})
    
    V[i, 5] = -basis.diff(reference_x).subs({reference_x: 1, reference_y: 0})
    V[i, 6] = (-basis.diff(reference_x).subs({reference_x: 1, reference_y: 0}) + +basis.diff(reference_y).subs({reference_x: 1, reference_y: 0}))
    
    V[i, 7] = -basis.diff(reference_y).subs({reference_x: 0, reference_y: 1})
    V[i, 8] = (+basis.diff(reference_x).subs({reference_x: 0, reference_y: 1}) + -basis.diff(reference_y).subs({reference_x: 0, reference_y: 1}))

    # center of mass
    V[i, 9] = basis.subs({reference_x: R(1,3), reference_y: R(1,3)})
    
mapping_basis = V.inv() @ monomial_basis

reference_v11_1 = reference_x_2 - reference_x_1
reference_v12_1 = reference_x_3 - reference_x_1
reference_v21_1 = reference_x_1 - reference_x_2
reference_v31_1 = reference_x_1 - reference_x_3

reference_v11_2 = reference_y_2 - reference_y_1
reference_v12_2 = reference_y_3 - reference_y_1
reference_v21_2 = reference_y_1 - reference_y_2
reference_v31_2 = reference_y_1 - reference_y_3


global_x = reference_x_1*mapping_basis[0] + reference_x_2*mapping_basis[1] + reference_x_1*mapping_basis[2] + \
    reference_v11_1*mapping_basis[3] + reference_v12_1*mapping_basis[4] + \
    reference_v21_1*mapping_basis[5] + reference_v22_1*mapping_basis[6] + \
    reference_v31_1*mapping_basis[7] + reference_v32_1*mapping_basis[8] + \
    R(1,3)*(reference_x_1 + reference_x_2 + reference_x_3)*mapping_basis[9]
    
    
global_y = reference_y_1*mapping_basis[0] + reference_y_2*mapping_basis[1] + reference_y_1*mapping_basis[2] + \
    reference_v11_2*mapping_basis[3] + reference_v12_2*mapping_basis[4] + \
    reference_v21_2*mapping_basis[5] + reference_v22_2*mapping_basis[6] + \
    reference_v31_2*mapping_basis[7] + reference_v32_2*mapping_basis[8] + \
    R(1,3)*(reference_y_1 + reference_y_2 + reference_y_3)*mapping_basis[9]
    

mapping_function = sympy.Matrix([global_x, global_y])


In [6]:
J_cofactor_T = mapping_function.jacobian([reference_x, reference_y]).cofactor_matrix().T

In [7]:
J = J_cofactor_T / J_cofactor_T.det()

In [8]:
u = sympy.Matrix([sympy.Function("u")(reference_x, reference_y)])
v = sympy.Matrix([sympy.Function("v")(reference_x, reference_y)])

In [9]:
grad_u = u.jacobian([reference_x, reference_y])@J_cofactor_T
grad_v = v.jacobian([reference_x, reference_y])@J_cofactor_T

weak_form = grad_u@grad_v.T

In [10]:
N = len(hermit_basis)

weak_form_functional = [[0 for i in range(N)] for j in range(N)]
weak_form_right_part = [[0 for i in range(N)] for j in range(N)]

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    first = hermit_basis[idx]
    second = hermit_basis[jdx]

    A = weak_form.subs({u[0]:first, v[0]:second}).doit()
    B = first * second
    
    weak_form_functional[idx][jdx] = A
    weak_form_right_part[idx][jdx] = B

weak_form_functional = sympy.Matrix(weak_form_functional) / J_cofactor_T.det()**2 * abs(J_cofactor_T.det())
weak_form_right_part = sympy.Matrix(weak_form_right_part) / J_cofactor_T.det()**2 * abs(J_cofactor_T.det())

  0%|          | 0/81 [00:00<?, ?it/s]

In [11]:
M = sympy.diag(R(1), 
               J.T.subs({reference_x:0, reference_y:0}), 
               R(1), 
               J.T.subs({reference_x:1, reference_y:0}), 
               R(1), 
               J.T.subs({reference_x:0, reference_y:1})
               ).inv().T

In [12]:
lambdify_symbols = [
    reference_x, reference_y,
    reference_x_1,
    reference_y_1,
    reference_x_2,
    reference_y_2,
    reference_x_3,
    reference_y_3,
    reference_v22_1,
    reference_v22_2,
    reference_v32_1,
    reference_v32_2,
]

In [13]:
weak_form_functional = M @ weak_form_functional @ M.T

In [14]:
weak_form_right_part = M @ weak_form_right_part @ M.T @ f_interpolation

In [15]:
weak_form_functional_lambdified = sympy.lambdify(lambdify_symbols, weak_form_functional, cse=True)

In [16]:
weak_form_right_part_lambdified = sympy.lambdify([*lambdify_symbols, *f_interpolation], weak_form_right_part, cse=True)

In [41]:
quadrature_weights = np.array([
0.090817990382754,
0.036725957756467,
0.036725957756467,
0.036725957756467,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.045321059435528,
0.045321059435528,
0.045321059435528,
]) / 2


quadrature_points = np.array([
[0.333333333333333,0.333333333333333],
[0.485577633383657,0.028844733232685],
[0.485577633383657,0.485577633383657],
[0.028844733232685,0.485577633383657],
[0.141707219414880,0.550352941820999],
[0.307939838764121,0.141707219414880],
[0.550352941820999,0.307939838764121],
[0.307939838764121,0.550352941820999],
[0.141707219414880,0.307939838764121],
[0.550352941820999,0.141707219414880],
[0.025003534762686,0.728323904597411],
[0.246672560639903,0.025003534762686],
[0.728323904597411,0.246672560639903],
[0.246672560639903,0.728323904597411],
[0.025003534762686,0.246672560639903],
[0.728323904597411,0.025003534762686],
[0.009540815400299,0.923655933587500],
[0.066803251012200,0.009540815400299],
[0.923655933587500,0.066803251012200],
[0.066803251012200,0.923655933587500],
[0.009540815400299,0.066803251012200],
[0.923655933587500,0.009540815400299],
[0.109481575485037,0.781036849029926],
[0.109481575485037,0.109481575485037],
[0.781036849029926,0.109481575485037],
])

In [47]:
def weak_form_functional_lambdified_quadrature(x_1, y_1, x_2, y_2, x_3, y_3, v_22_1, v_22_2, v_32_1, v_32_2):
    
    return weak_form_functional_lambdified(*quadrature_points.T, x_1, y_1, x_2, y_2, x_3, y_3, v_22_1, v_22_2, v_32_1, v_32_2)@quadrature_weights

In [60]:
def weak_form_right_part_lambdified_quadrature(x_1, y_1, x_2, y_2, x_3, y_3, v_22_1, v_22_2, v_32_1, v_32_2,
                                               f_1, f_2, f_3, f_4, f_5, f_6, f_7, f_8, f_9):
    
    return weak_form_right_part_lambdified(*quadrature_points.T, x_1, y_1, x_2, y_2, x_3, y_3, v_22_1, v_22_2, v_32_1, v_32_2,
                                           f_1, f_2, f_3, f_4, f_5, f_6, f_7, f_8, f_9)@quadrature_weights

In [17]:
interpolation_function = sympy.lambdify([*lambdify_symbols, *f_interpolation], ((M@hermit_basis).T@f_interpolation)[0, 0], cse=True)

In [18]:
mapping_function_lambdified = sympy.lambdify(lambdify_symbols, mapping_function[:, 0], cse=True)

In [ ]:
dill.dump(weak_form_functional_lambdified_quadrature, open("../calculations/cubic_hermite_weak_form_functional_cubic_hermite", "wb"))
dill.dump(weak_form_right_part_lambdified_quadrature, open("../calculations/cubic_hermite_weak_form_right_part_cubic_hermite", "wb"))
dill.dump(interpolation_function, open("../calculations/cubic_hermite_basis", "wb"))
dill.dump(mapping_function_lambdified, open("../calculations/cubic_hermite_mapping_function", "wb"))